In [4]:
import pandas as pd

In [5]:
data = pd.read_csv("./data/bmi_lbs.csv")
data['Weight(kg)'] = data['Weight(lbs)'] * 0.453592

#### 학습 데이터

In [6]:
X = data[['Height','Weight(kg)']]
y = data['Label']

#### 예측할 샘플 데이터

In [7]:
sample_data = data.iloc[[30,110,150,433,498],[2,4]]
ground_truth = data.iloc[[30,110,150,433,498],0]
sample_data

,Height,Weight(kg)
30,153,120.998660
110,150,69.999225
150,196,49.999446
433,179,126.998593
498,150,94.998948


#### 과제
 1. 예측 데이터와 모든 학습 데이터(500개)와의 거리 계산 -> 유클리드 거리 계산
 2. 계산된 모든 거리값 중 설정된 k값(k=5)만큼 가장 가까운 데이터 추출
 3. 추출된 데이터의 비율 확인 후 예측

In [8]:
X-sample_data.iloc[1]

,Height,Weight(kg)
0,24.0,25.999712
1,39.0,16.999812
2,35.0,39.999557
3,45.0,33.999623
4,-1.0,-8.999900
...,...,...
495,0.0,82.999081
496,34.0,50.999435
497,-9.0,65.999269
498,0.0,24.999723


In [7]:
# 1번
def my_KNN(input_data, k):
    # 카운터 딕셔너리
    from collections import Counter
    # 1. 모든 데이터와 input데이터 사이의 거리계산
    distance_list = []    # 결과는 5 X 500 크기
    for i in range(len(input_data)):
        temp = (X - input_data.iloc[i]) ** 2     # X데이터 전체[키, 몸무게] - i번째 샘플[키, 몸무게] -> 결과 : [a제곱, b제곱]
        distance = (temp.iloc[:, 0] + temp.iloc[:, 1])**0.5   # (키 + 몸무게) 의 제곱근
        distance_list.append(distance)
    distance_list = pd.Series(distance_list, index=input_data.index)   # list -> series 변환
    #print(distance_list)
    k_neighbors = []
    for i in range(len(input_data)):
        input_i = distance_list.iloc[i]    # 첫번째 샘플데이터(30번)
        temp = []
        for j in range(k):
            # k개만큼 최솟값들 뽑기
            idx = input_i.idxmin()
            temp.append(y.loc[idx])     # 구한 최솟값의 인덱스를 이용하여
            # 최솟값 중복 없애게 이번에 뽑은 최솟값은 제거해주기
            input_i.drop([idx], inplace=True)
        k_neighbors.append(temp)
    # k_neighbors -> 크기 : 5 X 5 (샘플데이터 갯수 X 5개의 최소거리 인덱스)
    #print(k_neighbor)
    # 3번
    output = []
    for nb in k_neighbors:
        count = Counter(nb)     # key : 등급, value: 등급이 나온 갯수
        output.append(max(count,key=count.get))
    output = pd.Series(output, index=input_data.index)
    return output

In [8]:
my_KNN(sample_data, 5)

30     Extreme Obesity
110            Obesity
150     Extremely Weak
433            Obesity
498    Extreme Obesity
dtype: object

In [9]:
#다른방법
def solution4(sample_data, X, y, k=5) -> list:
    
    answer = []
    for idx, val in sample_data.iterrows():
        distanceSer = X.apply(lambda x : dist(x, val), axis=1).sort_values()[1:1+k]
        
        SerLabel, SerVal = y.iloc[distanceSer.index], distanceSer.values
        
        countLabel = Counter(SerLabel.values)
        countCheck = max(countLabel, key=countLabel.get)
        
        if list(countLabel.values()).count(countLabel[countCheck]) < 2:
            answer.append((idx, countCheck))
        else:
            k_dict = {}
            
            for label, distance in zip(SerLabel.values, SerVal):
                if label not in k_dict.keys():
                    k_dict[label] = [distance]
                else:
                    k_dict[label].append(distance)
            
            dupliMaxVal = [(key, val) for key, val in k_dict.items() if len(val) == countLabel[countCheck]]
            meanCal = map(lambda x: (x[0], np.mean(x[1])), dupliMaxVal)
            
            answer.append((idx, min(meanCal, key=lambda x:x[1])[0]))
            
    return answer

In [10]:
#다른방법
import numpy as np
from math import dist
from collections import Counter
result =[]
for idx, val in sample_data.iterrows():
    cal = []
    tmp = []
    target = (val['Height'], val['Weight(kg)'])
    for idx2, val2 in data.iterrows():
        compare = (val2['Height'], val2['Weight(kg)'])
        if target != compare:
            cal.append((idx, idx2, dist(target, compare)))
    cal = sorted(cal, key=lambda x: x[2])[:5]
    for i in cal:
        tmp.append(data.iloc[i[1], 0])
    result.append(Counter(tmp).most_common()[0][0])
display(result)

['Extreme Obesity', 'Obesity', 'Extremely Weak', 'Obesity', 'Extreme Obesity']

In [11]:
#다른방법
from math import dist
from collections import Counter
dest = []
for idx, val in sample_data.iterrows(): # 예측값/ 행단위로 하나씩 뽑음 iterrows
    target = ((val['Height'], val['Weight(kg)'])) # 키,몸무게를 각각 x,y로 담고, x,y의 묶음을 튜플로 저장.
    calLi = []
    for idx2, val2 in data.iterrows():
        compare = ((val2['Height'], val2['Weight(kg)'])) #(120, 50)
        if target != compare: #예측할 값과 전체데이터의 값이 같지 않을때 / 자기자신과 중복되지않게
            calLi.append((idx, idx2, dist(target, compare)))
            # dist -> a,b의 거리를 계산해주는 함수 / a(x,y) b(x,y)
            # calLi에는 499개가 담긴다~
    doyouknow = [(i[1], i[2]) for i in sorted(calLi, key= lambda x:x[2])[:5]]
    #calLi의 dist값을 기준으로 정렬하고 5개만 가져와서 index와 dist 값을 저장
    doyouknowLabel = data.iloc[[i[0] for i in doyouknow], [0]]["Label"]
    #data의 인덱스와 매칭시켜 라벨 가져오기 / 시리즈데이터
    doyouknowResult = list(zip(doyouknowLabel.values, [i[1] for i in doyouknow]))
    #라벨 / 거리값 묶어준다
    mostCommLabel = Counter([i[0] for i in doyouknowResult])
    #Counter 중복된 데이터가 저장된 배열을 인자로 넘기면 각 원소가 몇 번씩 나오는지가 저장된 객체를 얻게 됩니다.
    #라벨값이 몇개있는지 센다
    print(mostCommLabel.most_common(1)[0][0])
    #most_common 수가 많은 순으로 정렬된 튜플 배열 리스트를 리턴

Extreme Obesity
Obesity
Extremely Weak
Obesity
Extreme Obesity


In [13]:
target

(150.0, 94.998947704)

In [2]:
import numpy as np

In [18]:
a=np.array([3,10,4,31,5,2,40,20])
b=np.argsort(a)[::-1]
a[b[:5]]

array([40, 31, 20, 10,  5])